# Bonus Notebook: Create a Dialect Specific Translation to the schema.org Vocabulary Words Google Recommends for Datasets and create JSON-LD 
### Notebook Goals
* Create dataframe with xpath content for the collection 
* Identify the Xpaths that Translate into the schema.org vocabulary
* Create valid JSON-LD for a record
* Use Google's Structured Data Testing Tool to test results
* Additional food for thought

In [ ]:
# refine dataframe, create record json
import pandas as pd
# create dataframe structure that contains the records content
import MDeval as md

#### Describe the metadata. 
* What organization created the records? (Organization)
* What collection are the records from? (Collection)
* What dialect are the records written in? (Dialect)

In [ ]:
# variables for function arguments, fill these out
Organization = 'LTER'
Collection = 'MILES'
Dialect = 'EML'

#### Read in the metadata's xpath evaluated csv

In [ ]:
# read in the Xpath evaluated csv defined by the above variables
RecommendationEvaluatedDF = pd.read_csv(
    './data/'+Organization+'/'+Collection+'_'+Dialect+'_ElementEvaluated.csv'
)

#### Record Xpath Content Function
* Rows are records
* Columns are Xpaths

In [ ]:
''' requires a dataframe with concepts. Creates a vertical view of
xpath content for each record in the collection. Useful in the
creation of json. 
'''
recordDF = md.recordXpathContent(RecommendationEvaluatedDF)
# display the dataframe
recordDF

#### Choose a record to translate

In [ ]:
# Set RecordChoice variable
RecordChoice = 'dataset_3484.xml'

In [ ]:
# Select record row
recordDF = recordDF[recordDF['Record'] == RecordChoice]
# drop the Collection and Record columns
recordDF = recordDF.drop(['Collection', 'Record'], 'columns')
# display the chosen record's content
recordDF

#### Translate xpaths to schema.org vocabulary
* identify xpaths that crosswalk
* replace 'xpathN' with Xpath from dialect

In [ ]:
# consider the schema.org vocabulary words and identify likely Xpath matches
recordDF = recordDF.rename({'xpath0':'name', 'xpath1':'description',
                            'xpath2':'url', 'xpath3':'keywords',
                            'xpath4':'creator', 'xpath5': 'distribution',
                            'xpath6':'@type', 'xpath7': 'version',
                            'xpath8': 'temporalCoverage', 'xpath9': 'spatialCoverage',
                            'xpath10':'citation'}, axis='columns')
# display the new dataframe
recordDF

#### Add the required context

In [ ]:
'''
insert a column in the third position with "@context" as the header
and "http://schema.org as the content
'''
recordDF.insert(2, '@context', 'http://schema.org/')
# display the dataframe
recordDF

#### Create JSON-LD string for adding to the header of the landing page in the repository

In [ ]:
# create JSON using the pandas function "to_json" use the "record" orientation
recordJSON = recordDF.to_json(orient='records')
# add the HTML script element that declares the contents to be JSON-LD
RecordJSONld = '<script type="application/ld+json">' + recordDict[1:-1] + '</script>'
# display the resultant string
RecordJSONld

#### Test JSON-LD for validity
* Take string produced by the cell above and copy it
* Go to [Google's Structured Data Testing Tool](https://search.google.com/structured-data/testing-tool#new-test)
* Select the "Code Snippet Tab"
* Paste string and "Run Test"
* Click on errors to highlight the portion of the string that needs improvement
* Rerun test with the play button in the middle bottom of the screen

#### Consider using another JSON writer

* Some of the schema.org vocabulary used in the recommendation for a dataset is nested.
* [hone](https://github.com/chamkank/hone) can create a nested JSON representation.


In [ ]:
# needed to create nest JSON
import hone
#requires a csv to read, so we'll need to save the 

# define csv location
toNestedJSON = (
    './data/' + Organization + '/' + Collection +
    '_' + Dialect + '_toNestedJSON.csv')
# change xpath headers in dataframe to delimiters used to nest
recordDF.columns = recordDF.columns.str.replace("[/]", " ")
# write results to csv
recordDF.to_csv(
    toNestedJSON, index=False
)
# write JSON
Hone = hone.Hone()
schema = Hone.get_schema(toNestedJSON)
# display JSON
schema

#### Consider creating a validation workflow for your JSON output from tools such as:

* https://github.com/digitalbazaar/pyld
* https://github.com/RDFLib/rdflib-jsonld

#### Improve workflow for specific dialect needs and share the resulting notebook(s) in the workshop_shared directory in the scgordon/participants directory to facilitate collaboration!